In [1]:
import tensorflow as tf
from tensorflow import keras

import utils

In [2]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = utils.load_housing_data()

## Define custom loss class

In [3]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = tf.abs(error) - 0.5
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super(HuberLoss, self).get_config()
        return {
            **base_config,
            'threshold': self.threshold
        }

## Training with custom loss

In [4]:
HUBER_LOSS_THRESHOLD = 2.0

In [12]:
utils.reset_session()

model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=x_train.shape[1:]),
    keras.layers.Dense(1),
])

model.compile(loss=HuberLoss(HUBER_LOSS_THRESHOLD), optimizer='adam', metrics=['mae'])

history = model.fit(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

Epoch 1/5
363/363 [==============================] - 0s 810us/step - loss: 0.3875 - mae: 0.7219 - val_loss: 0.2596 - val_mae: 0.5525
Epoch 2/5
363/363 [==============================] - 0s 647us/step - loss: 0.2090 - mae: 0.4944 - val_loss: 0.1962 - val_mae: 0.4688
Epoch 3/5
363/363 [==============================] - 0s 650us/step - loss: 0.1850 - mae: 0.4597 - val_loss: 0.1758 - val_mae: 0.4392
Epoch 4/5
363/363 [==============================] - 0s 694us/step - loss: 0.1767 - mae: 0.4477 - val_loss: 0.1633 - val_mae: 0.4267
Epoch 5/5
363/363 [==============================] - 0s 675us/step - loss: 0.1717 - mae: 0.4397 - val_loss: 0.1688 - val_mae: 0.4338


In [13]:
model.evaluate(x_val, y_val)

121/121 [==============================] - 0s 438us/step - loss: 0.1688 - mae: 0.4338


[0.1688012182712555, 0.4338049590587616]

## Saving/Loading custom loss 

In [7]:
model.save('model.h5')

In [8]:
utils.reset_session()
try:
    model = keras.models.load_model('model.h5')
except ValueError as e:
    print(e)

Unknown loss function: HuberLoss


In [9]:
utils.reset_session()

model = keras.models.load_model('model.h5', custom_objects={'HuberLoss': HuberLoss})
model.evaluate(x_val, y_val)

121/121 [==============================] - 0s 364us/step - loss: 0.1742 - mean_absolute_error: 0.4341


[0.17419719696044922, 0.43409448862075806]

In [10]:
history = model.fit(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Epoch 1/2
363/363 [==============================] - 0s 677us/step - loss: 0.1766 - mean_absolute_error: 0.4390 - val_loss: 0.1661 - val_mean_absolute_error: 0.4231
Epoch 2/2
363/363 [==============================] - 0s 625us/step - loss: 0.1734 - mean_absolute_error: 0.4345 - val_loss: 0.1783 - val_mean_absolute_error: 0.4377
